In [ ]:
from google.colab import drive
import glob
import zipfile

# ----------------------------------------------------------------------
# 🚨 초기화 섹션: 새 파일을 처리하기 위해 이전 데이터를 모두 초기화합니다. 🚨
# ----------------------------------------------------------------------
# 1. 이전 파일 목록 초기화
json_files = []
# 2. 이전 데이터프레임 리스트 초기화
dataframes = []
# 3. 이전 파일-DF 맵핑 초기화
file_to_df_map = {}

#data 폴더 지우기
!rm -rf /content/data/*
# ----------------------------------------------------------------------

# 1. 구글 드라이브 마운트
drive.mount('/content/drive')

# 2. 데이터가 저장된 경로 지정
root = '/content/drive/MyDrive'

# 3. 데이터 불러오기
zip_path = '/content/drive/MyDrive/SAMPLE_label.zip'  # 실제 경로로 변경
extract_path = '/content/data'  # 압축을 풀 위치

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("압축 해제 완료!")

import glob

json_files = glob.glob(f"{extract_path}/**/*.json", recursive=True)
print(f"JSON 파일 개수: {len(json_files)}개")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
압축 해제 완료!
JSON 파일 개수: 100개


In [ ]:
import pandas as pd
import json
import os

# --- 2. 데이터 처리 함수 정의 (파일 레벨 메타데이터 및 _id 추출) ---

def extract_metadata_with_id(data):
    """
    단일 통화 JSON 데이터에서 요청된 메타데이터와 파일 _id를 추출하여 단일 행을 반환합니다.
    """
    # 🚨 수정된 부분: _id를 'id'로 사용하고 recordId도 포함 🚨
    id_unique = data.get("_id")       # JSON 파일 자체의 고유 ID (_id)
    record_id = data.get("recordId")  # 통화 녹음의 고유 ID

    # 요청된 메타데이터 추출
    media_type = data.get("mediaType")
    gender = data.get("gender")
    address = data.get("address")
    disaster_large = data.get("disasterLarge")
    disaster_medium = data.get("disasterMedium")
    urgency_level = data.get("urgencyLevel")
    sentiment = data.get("sentiment")

    # 🚨 수정 완료된 부분: symptom 처리 (빈 리스트 [] 처리) 🚨
    symptom_list = data.get("symptom")
    if symptom_list is None or (isinstance(symptom_list, list) and not symptom_list):
        symptom = "해당 없음"
    elif isinstance(symptom_list, list):
        symptom = ", ".join(symptom_list)
    else: # 리스트가 아니면서 빈 값이 아닌 경우 (문자열 등)
        symptom = str(symptom_list)

    # 🚨 수정 완료된 부분: triage 처리 (빈 문자열 "" 처리) 🚨
    triage_value = data.get("triage")
    triage = "해당 없음" if not triage_value else triage_value

    # 단일 파일당 단일 행(Row) 생성
    row = {
        "id": id_unique,                # 요청에 따라 _id 값을 'id'로 사용
        "recordId": record_id,          # 통화 녹음 ID
        "mediaType": media_type,
        "gender": gender,
        "address": address,
        "disasterLarge": disaster_large,
        "disasterMedium": disaster_medium,
        "urgencyLevel": urgency_level,
        "sentiment": sentiment,
        "symptom": symptom,
        "triage": triage
    }

    # DataFrame 생성을 위해 단일 행을 리스트로 반환
    return [row]

# --- 3. JSON 파일 목록 순회 및 개별 DataFrame 생성 ---

# 생성된 모든 DataFrame을 저장할 리스트
dataframes = []
file_to_df_map = {} # 파일 이름과 DataFrame을 매핑할 딕셔너리 (접근 용이)

if json_files:
    print("\n⏳ 모든 JSON 파일을 읽고 개별 DataFrame (1행)을 생성합니다...")
    for file_path in json_files:
        file_name = os.path.basename(file_path)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # 파일의 메타데이터 행을 추출 (1행)
            rows = extract_metadata_with_id(data)

            # 1. 하나의 파일 내용으로 DataFrame 생성 (1행짜리 DataFrame)
            df_single = pd.DataFrame(rows)

            # 2. DataFrame 리스트에 추가
            dataframes.append(df_single)

            # 3. 파일 이름으로 DataFrame에 접근할 수 있도록 딕셔너리에 저장
            file_to_df_map[file_name] = df_single

        except Exception as e:
            print(f"❌ 데이터 처리 중 에러 발생 파일: {file_name}, 에러: {e}")

    # 결과를 확인하기 위해 생성된 DataFrame의 수와 첫 번째 DataFrame의 정보를 출력
    print("\n" + "="*50)
    print(f"🎉 총 {len(dataframes)}개의 DataFrame 생성 완료!")
    print(" (각 DataFrame은 1개의 행과 ID 컬럼들을 포함합니다.)")
    print("="*50)

    if dataframes:
        print("\n📌 첫 번째 DataFrame 미리보기 (_id 값이 'id' 컬럼에 포함됨):")
        df_first = dataframes[0]
        # 인덱스 없이 깔끔하게 출력
        print(df_first.to_string(index=False))
        display(df_first)
        print("-" * 50)
        print(f"📌 첫 번째 DataFrame 형태: {df_first.shape}")
        print("\n*주의: 모든 DataFrame은 'dataframes' 리스트와 'file_to_df_map' 딕셔너리에 개별적으로 저장되어 있습니다.")

        #display(df_first)
else:
    print("\n⚠️ 처리할 JSON 파일이 없어 DataFrame이 생성되지 않았습니다.")


⏳ 모든 JSON 파일을 읽고 개별 DataFrame (1행)을 생성합니다...

🎉 총 100개의 DataFrame 생성 완료!
 (각 DataFrame은 1개의 행과 ID 컬럼들을 포함합니다.)

📌 첫 번째 DataFrame 미리보기 (_id 값이 'id' 컬럼에 포함됨):
                      id             recordId mediaType gender       address disasterLarge disasterMedium urgencyLevel sentiment symptom triage
651e464d69a4f266f0626a0e 6cbeb963c82b7c7f6c91    Mobile      M 서울특별시 관악구 신림동            구조           안전사고            중     당황/난처      염좌 잠재응급증상


,id,recordId,mediaType,gender,address,disasterLarge,disasterMedium,urgencyLevel,sentiment,symptom,triage
0,651e464d69a4f266f0626a0e,6cbeb963c82b7c7f6c91,Mobile,M,서울특별시 관악구 신림동,구조,안전사고,중,당황/난처,염좌,잠재응급증상


--------------------------------------------------
📌 첫 번째 DataFrame 형태: (1, 11)

*주의: 모든 DataFrame은 'dataframes' 리스트와 'file_to_df_map' 딕셔너리에 개별적으로 저장되어 있습니다.


In [ ]:

OUTPUT_CSV_PATH = '/content/drive/MyDrive/sample_meta_data.csv' # 🚨 최종 CSV 저장 경로 🚨

# 최종 DataFrame 생성
if dataframes:
        final_df = pd.concat(dataframes, ignore_index=True)
        print(f"\n🎉 최종 DataFrame 생성 완료! (총 {final_df.shape[0]} 행)")
else:
        final_df = None
        print("\n⚠️ 처리된 데이터가 없어 최종 DataFrame을 생성하지 못했습니다.")

# --- 4. 최종 DataFrame을 CSV 파일로 저장 ---

if final_df is not None:
    try:
        # 인덱스(순번)를 제외하고 CSV로 저장합니다.
        final_df.to_csv(OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')

        print("\n" + "="*70)
        print(f"✅ CSV 파일 저장 성공!")
        print(f"저장 경로: {OUTPUT_CSV_PATH}")
        print("="*70)

    except Exception as e:
        print(f"\n❌ CSV 파일 저장 중 오류 발생: {e}")


🎉 최종 DataFrame 생성 완료! (총 100 행)

✅ CSV 파일 저장 성공!
저장 경로: /content/drive/MyDrive/sample_meta_data.csv


In [ ]:
import pandas as pd
import numpy as np
import math

# 이 코드는 이전 단계에서 'final_df'라는 이름의 통합 DataFrame이 성공적으로 생성되었음을 가정합니다.

# --- 1. 범주형 컬럼 정의 ---
# 'id'와 'recordId'는 고유성이 너무 높아 임베딩 대신 다른 처리가 필요하므로 제외합니다.
CATEGORICAL_COLS = [
    'mediaType', 'gender', 'disasterLarge', 'disasterMedium',
    'urgencyLevel', 'sentiment', 'symptom', 'triage', 'address'
]

# 실제로 final_df에 존재하는 범주형 컬럼만 필터링합니다.
existing_cat_cols = [col for col in CATEGORICAL_COLS if col in final_df.columns]
if not existing_cat_cols:
    print("❌ 오류: DataFrame에 임베딩할 범주형 컬럼이 없습니다.")
    exit()

# --- 2. 임베딩 차원 계산 규칙 정의 ---
def calculate_embedding_dim(cardinality):
    """
    고유 값 개수(Cardinality)를 기반으로 임베딩 차원을 계산하는 함수입니다.

    규칙:
    1. 고유 값 개수가 매우 적으면 (예: 5개 이하) 차원 2를 사용합니다.
    2. 일반적으로 log2(C)나 C**0.25 규칙을 사용하며, 차원이 너무 커지지 않도록 최대 50으로 제한합니다.
    """
    if cardinality <= 5:
        return 2

    # 규칙 1: log2(Cardinality) + 1 (컴퓨터 비전 분야에서 흔히 사용)
    dim_log = math.ceil(math.log2(cardinality)) + 1

    # 규칙 2: Cardinality의 네제곱근 (C**0.25)
    dim_root = math.ceil(cardinality ** 0.25)

    # 두 규칙 중 작은 값을 사용하고, 최소 4, 최대 50으로 제한합니다.
    return max(4, min(dim_log, dim_root, 50))


# --- 3. 각 컬럼별 임베딩 차원 계산 및 결과 출력 ---

embedding_dims = {}
vocab_sizes = {} # nn.Embedding에 필요한 고유 값 개수 (어휘 크기)

print("🔍 범주형 컬럼별 고유 값 개수(Cardinality) 및 임베딩 차원 계산 결과:")
print("-" * 70)
print(f"{'컬럼명':<20} | {'고유 값 개수 (C)':<15} | {'임베딩 차원 (D)':<15} | {'규칙 (max(4, min(log2(C)+1, C**0.25, 50)))':<15}")
print("-" * 70)

for col in existing_cat_cols:
    # 결측치(NaN)를 제외한 고유 값 개수를 계산합니다.
    cardinality = final_df[col].nunique()

    # PyTorch의 nn.Embedding은 0을 패딩(Padding) 토큰으로 사용하기 위해
    # 고유 값 개수에 1을 더한 값을 어휘 크기로 사용합니다.
    vocab_size = cardinality + 1

    # 임베딩 차원 계산
    emb_dim = calculate_embedding_dim(cardinality)

    embedding_dims[col] = emb_dim
    vocab_sizes[col] = vocab_size

    print(f"{col:<20} | {cardinality:<15} | {emb_dim:<15} | {emb_dim if cardinality > 5 else 2}")

print("-" * 70)

# 최종 결과 딕셔너리 출력
print("\n✅ 임베딩 차원 딕셔너리 (embedding_dims):")
print(embedding_dims)
print("\n✅ 어휘 크기 딕셔너리 (vocab_sizes, nn.Embedding 입력값):")
print(vocab_sizes)

In [ ]:
import pandas as pd
import os
from google.colab import drive

# 1. Google Drive 마운트
drive.mount('/content/drive')

# 🚨 파일 경로를 실제 CSV 파일 경로로 변경해 주세요.
FILE_PATH_1 = '/content/drive/MyDrive/df_vl.csv' # '_id' 컬럼이 있는 파일
FILE_PATH_2 = '/content/drive/MyDrive/VL_meta_data_output.csv'  # 'id' 컬럼이 있는 파일

# 🚨 병합된 결과를 저장할 경로를 지정합니다.
OUTPUT_MERGED_CSV_PATH = '/content/drive/MyDrive/VL_merged_meta_data.csv'

try:
    # 2. 두 CSV 파일을 각각 DataFrame으로 불러옵니다.
    df_left = pd.read_csv(FILE_PATH_1)
    df_right = pd.read_csv(FILE_PATH_2)

    print(f"✅ 파일 1 ({os.path.basename(FILE_PATH_1)}) 로드 완료. 형태: {df_left.shape}")
    print(f"✅ 파일 2 ({os.path.basename(FILE_PATH_2)}) 로드 완료. 형태: {df_right.shape}")

# 3. 컬럼 이름 통일: df_left의 '_id'를 'id'로 변경합니다.
    if '_id' in df_left.columns:
        df_left.rename(columns={'_id': 'id'}, inplace=True)
        print("💡 FILE_PATH_1의 '_id' 컬럼을 'id'로 변경했습니다.")
    elif 'id' not in df_left.columns:
         raise ValueError("FILE_PATH_1에 '_id' 또는 'id' 컬럼이 존재하지 않습니다.")

    # 4. 'id' 컬럼을 기준으로 두 DataFrame을 병합합니다.
    # how='inner'를 사용하여 두 파일 모두에 존재하는 ID만 병합합니다.
    merged_df = pd.merge(
        df_left,
        df_right,
        on='id',
        how='inner'
    )

    print("-" * 50)
    print("🎉 ID를 기준으로 데이터 병합 완료!")
    print(f"최종 DataFrame 형태: {merged_df.shape}")
    print("-" * 50)
    print("📌 병합된 DataFrame 미리보기:")
    print(merged_df.head())
    print("-" * 50)

    # 5. 병합된 결과를 새로운 CSV 파일로 저장합니다.
    merged_df.to_csv(OUTPUT_MERGED_CSV_PATH, index=False, encoding='utf-8-sig')

    print(f"✅ 최종 CSV 파일 저장 성공!")
    print(f"저장 경로: {OUTPUT_MERGED_CSV_PATH}")

except FileNotFoundError as e:
    print(f"❌ 파일 로드 오류: 지정된 경로에 CSV 파일이 없습니다. 경로를 확인해 주세요: {e.filename}")
except Exception as e:
    print(f"❌ 병합 중 예상치 못한 오류 발생: {e}")

아래는 테스트 데이터(전체보기)

In [ ]:
import pandas as pd
import json
import os

# --- 2. 데이터 처리 함수 정의 (파일 레벨 메타데이터 및 _id 추출) ---

def extract_metadata_with_id(data):
    """
    단일 통화 JSON 데이터에서 요청된 메타데이터와 파일 _id를 추출하여 단일 행을 반환합니다.
    """
    # 🚨 수정된 부분: _id를 'id'로 사용하고 recordId도 포함 🚨
    id_unique = data.get("_id")       # JSON 파일 자체의 고유 ID (_id)
    record_id = data.get("recordId")  # 통화 녹음의 고유 ID

    # 요청된 메타데이터 추출
    media_type = data.get("mediaType")
    gender = data.get("gender")
    address = data.get("address")
    disaster_large = data.get("disasterLarge")
    disaster_medium = data.get("disasterMedium")
    urgency_level = data.get("urgencyLevel")
    sentiment = data.get("sentiment")

    # 🚨 수정 완료된 부분: symptom 처리 (빈 리스트 [] 처리) 🚨
    symptom_list = data.get("symptom")
    if symptom_list is None or (isinstance(symptom_list, list) and not symptom_list):
        symptom = "해당 없음"
    elif isinstance(symptom_list, list):
        symptom = ", ".join(symptom_list)
    else: # 리스트가 아니면서 빈 값이 아닌 경우 (문자열 등)
        symptom = str(symptom_list)

    # 🚨 수정 완료된 부분: triage 처리 (빈 문자열 "" 처리) 🚨
    triage_value = data.get("triage")
    triage = "해당 없음" if not triage_value else triage_value

    # 단일 파일당 단일 행(Row) 생성
    row = {
        "id": id_unique,                # 요청에 따라 _id 값을 'id'로 사용
        "recordId": record_id,          # 통화 녹음 ID
        "mediaType": media_type,
        "gender": gender,
        "address": address,
        "disasterLarge": disaster_large,
        "disasterMedium": disaster_medium,
        "urgencyLevel": urgency_level,
        "sentiment": sentiment,
        "symptom": symptom,
        "triage": triage
    }

    # DataFrame 생성을 위해 단일 행을 리스트로 반환
    return [row]

# --- 3. JSON 파일 목록 순회 및 통합 DataFrame 생성 ---

dataframes = []

if json_files:
    print("\n⏳ 모든 JSON 파일을 읽고 메타데이터를 통합 DataFrame으로 합칩니다...")
    for file_path in json_files:
        file_name = os.path.basename(file_path)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # 파일의 메타데이터 행을 추출 (1행)
            rows = extract_metadata_with_id(data)

            # 1행짜리 DataFrame 생성 후 리스트에 추가
            df_single = pd.DataFrame(rows)
            dataframes.append(df_single)

        except Exception as e:
            print(f"❌ 데이터 처리 중 에러 발생 파일: {file_name}, 에러: {e}")

    # --- 4. 모든 1행짜리 DataFrame을 하나의 최종 DataFrame으로 합치기 ---

    if dataframes:
        # 모든 개별 DataFrame을 하나로 합쳐 여러 행으로 구성합니다.
        final_df = pd.concat(dataframes, ignore_index=True)

        print("\n" + "="*70)
        print("🎉 모든 파일의 메타데이터가 통합된 DataFrame 생성 완료 (여러 행)")
        print("="*70)
        print(f"총 파일(행) 개수: {final_df.shape[0]}개")
        print(f"총 컬럼 개수: {final_df.shape[1]}개")
        print("-" * 70)

        # 전체 데이터 확인을 위해 출력 설정을 조정
        pd.set_option('display.max_rows', 500)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 1000)

        print("\n📌 통합 DataFrame 전체 내용 (여러 행):")
        # 모든 행을 출력합니다.
        print(final_df.to_string())

        print("-" * 70)
        print("\n📌 컬럼 및 데이터 타입 정보:")
        print(final_df.info())

else:
    print("\n⚠️ 처리할 JSON 파일이 없어 DataFrame이 생성되지 않았습니다.")